<a href="https://colab.research.google.com/github/SahilDhull/emphasis_selection/blob/master/model/bert_as_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers
!pip install config

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForMaskedLM , BertModel ,WEIGHTS_NAME, AdamW
from transformers import PreTrainedModel, PreTrainedTokenizer
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import codecs

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

train_file = 'drive/My Drive/datasets/train.txt'
dev_file = 'drive/My Drive/datasets/dev.txt'

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [0]:
def read_token_map(file,word_index = 1,prob_index = 4, caseless = True):
  
  with codecs.open(file, 'r', 'utf-8') as f:
      lines = f.readlines()

  tokenized_texts = []
  token_map = []
  token_labels = []

  bert_tokens = []
  orig_to_tok_map = []
  labels = []

  bert_tokens.append("[CLS]")
  
  for line in lines:
    if not (line.isspace()):
      feats = line.strip().split()
      word = feats[word_index].lower() if caseless else feats[word_ind]
      label = feats[prob_index].lower() if caseless else feats[word_ind]
      labels.append((float)(label))
      orig_to_tok_map.append(len(bert_tokens))
      bert_tokens.extend(tokenizer.tokenize(word))
    elif len(orig_to_tok_map) > 0:
      bert_tokens.append("[SEP]")
      tokenized_texts.append(bert_tokens)
      token_map.append(orig_to_tok_map)
      token_labels.append(labels)
      bert_tokens = []
      orig_to_tok_map = []
      labels = []
      bert_tokens.append("[CLS]")
          
  if len(orig_to_tok_map) > 0:
    bert_tokens.append("[SEP]")
    tokenized_texts.append(bert_tokens)
    token_map.append(orig_to_tok_map)
    token_labels.append(labels)
  
  return tokenized_texts, token_map, token_labels

In [0]:
t_tokenized_texts, t_token_map, t_token_label = read_token_map(train_file)
print(t_tokenized_texts[100])
print(t_token_map[100])
print(t_token_label[100])

d_tokenized_texts, d_token_map, d_token_label = read_token_map(dev_file)
print(d_tokenized_texts[50])
print(d_token_map[50])
print(d_token_label[50])

In [0]:
MAX_LEN = 36

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
t_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in t_tokenized_texts]

# Pad our input tokens
t_input_ids = pad_sequences(t_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
t_token_map = pad_sequences(t_token_map, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
t_token_label = pad_sequences(t_token_label, maxlen=MAX_LEN, dtype="float", truncating="post", padding="post")

print(t_input_ids[100])
print(t_token_map[100])
print(t_token_label[100])


In [0]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
d_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in d_tokenized_texts]

# Pad our input tokens
d_input_ids = pad_sequences(d_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
d_token_map = pad_sequences(d_token_map, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
d_token_label = pad_sequences(d_token_label, maxlen=MAX_LEN, dtype="float", truncating="post", padding="post")

print(d_input_ids[50])
print(d_token_map[50])
print(d_token_label[50])

In [0]:
t_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in t_input_ids:
  seq_mask = [float(i>0) for i in seq]
  t_attention_masks.append(seq_mask)
print(t_attention_masks[100])

d_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in d_input_ids:
  seq_mask = [float(i>0) for i in seq]
  d_attention_masks.append(seq_mask)
print(d_attention_masks[50])

In [0]:
# train_inputs, validation_inputs = train_test_split(input_ids, random_state=2018, test_size=0.1)
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
#                                              random_state=2018, test_size=0.1)
                                             
# Convert all of our data into torch tensors, the required datatype for our model
t_input_ids = torch.tensor(t_input_ids)
t_token_map = torch.tensor(t_token_map )
t_token_label = torch.tensor(t_token_label)
t_attention_masks = torch.tensor(t_attention_masks)

d_input_ids = torch.tensor(d_input_ids)
d_token_map = torch.tensor(d_token_map )
d_token_label = torch.tensor(d_token_label)
d_attention_masks = torch.tensor(d_attention_masks)

# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(t_input_ids, t_token_map, t_token_label, t_attention_masks)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(d_input_ids, d_token_map, d_token_label, d_attention_masks)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
model = BertModel.from_pretrained("bert-base-uncased")
model.cuda()

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-5, warmup=.1)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 4

# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
  # Set our model to training mode
  model.train()  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    print(b_input_ids.size(),"\n\n", b_input_mask.size())
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    hidden_rep , cls_head = model(b_input_ids, attention_mask=b_input_mask)
    print(cls_head.size())
    print(len(hidden_rep))
    cls_rep = hidden_rep[0]

    print(cls_rep.size())
  #   train_loss_set.append(loss.item())    
  #   # Backward pass
  #   loss.backward()
  #   # Update parameters and take a step using the computed gradient
  #   optimizer.step()
  #   # Update tracking variables
  #   tr_loss += loss.item()
  #   nb_tr_examples += b_input_ids.size(0)
  #   nb_tr_steps += 1
  # print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
#   ## VALIDATION

#   # Put model in evaluation mode
#   model.eval()
#   # Tracking variables 
#   eval_loss, eval_accuracy = 0, 0
#   nb_eval_steps, nb_eval_examples = 0, 0
#   # Evaluate data for one epoch
#   for batch in validation_dataloader:
#     # Add batch to GPU
#     batch = tuple(t.to(device) for t in batch)
#     # Unpack the inputs from our dataloader
#     b_input_ids, b_input_mask = batch
#     # Telling the model not to compute or store gradients, saving memory and speeding up validation
#     with torch.no_grad():
#       # Forward pass, calculate logit predictions
#       logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
#     # Move logits and labels to CPU
#     logits = logits.detach().cpu().numpy()
#     label_ids = b_labels.to('cpu').numpy()
#     tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
#     eval_accuracy += tmp_eval_accuracy
#     nb_eval_steps += 1
#   print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

# # plot training performance
# plt.figure(figsize=(15,8))
# plt.title("Training loss")
# plt.xlabel("Batch")
# plt.ylabel("Loss")
# plt.plot(train_loss_set)
# plt.show()

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)